# Idea


- We treat every single FVC measurement as if it were a "baseline" measurement. This increases the number of training samples from 1549 to 12144
- The week of that baseline is base_Week and the associated FVC is base_FVC along with base_Percent
- We calculate the number of weeks since that "baseline" in base_Week, to create Week_passed
- Our aim is to build a LGBM model that can predict FVC in any week if you are given a baseline. This means that when predicting on test, we can provide a list of weeks to generate predictions
- The features we use are: base_FVC, base_Percent, base_Age, Week_passed, Sex, SmokingStatus
- Once we have FVC predictions, we build a second LGBM model to predict confidence, which we create labels for in cell 14 based on how inaccurate our FVC predictions were

# Import packages

In [30]:
# !pip install category_encoders
# !pip install lightgbm
import os
from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
from functools import partial
from sklearn import preprocessing
import numpy as np
import pandas as pd
import random
import math

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.metrics import mean_squared_error
import category_encoders as ce

from PIL import Image
import cv2
import pydicom

import torch

import lightgbm as lgb
from sklearn.linear_model import Ridge

import warnings
warnings.filterwarnings("ignore")

# !pip install hyperopt
import numpy as np
import pandas as pd
from functools import partial
import xgboost as xgb
from numpy.ma import MaskedArray
import sklearn.utils.fixes
sklearn.utils.fixes.MaskedArray = MaskedArray

from sklearn import metrics
from sklearn import ensemble
from sklearn import model_selection
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope

# Utils

In [31]:
def get_logger(filename='log'):
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

logger = get_logger()


def seed_everything(seed=777):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# Config

In [32]:
OUTPUT_DICT = './'

ID = 'Patient_Week'
TARGET = 'FVC'
SEED = 42
seed_everything(seed=SEED)

N_FOLD = 4

# Data Prep

## Loading

In [33]:
train = pd.read_csv("../input/osic-pulmonary-fibrosis-progression/train.csv")
train[ID] = train['Patient'].astype(str) + "_" + train["Weeks"].astype(str)
print(train.shape)
train.head()

(1549, 8)


,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,Patient_Week
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker,ID00007637202177411956430_-4
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker,ID00007637202177411956430_5
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker,ID00007637202177411956430_7
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker,ID00007637202177411956430_9
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker,ID00007637202177411956430_11


## Construct training input 

In [34]:
output = pd.DataFrame()
gb = train.groupby('Patient')
tk0 = tqdm(gb, total=len(gb)) 
for _, usr_df in tk0:
    usr_output = pd.DataFrame()
    for week, tmp in usr_df.groupby('Weeks'):
        rename_cols = {'Weeks': 'base_Week', 'FVC': 'base_FVC', 'Percent': 'base_Percent', 'Age': 'base_Age'}
        tmp = tmp.drop(columns='Patient_Week').rename(columns=rename_cols)
        drop_cols = ['Age', 'Sex', 'SmokingStatus', 'Percent']
        _usr_output = usr_df.drop(columns=drop_cols).rename(columns={'Weeks': 'predict_Week'}).merge(tmp, on='Patient')
        _usr_output['Week_passed'] = _usr_output['predict_Week'] - _usr_output['base_Week']
        usr_output = pd.concat([usr_output, _usr_output])
    output = pd.concat([output, usr_output])

print(output.shape)
output.head()


(13707, 11)


,Patient,predict_Week,FVC,Patient_Week,base_Week,base_FVC,base_Percent,base_Age,Sex,SmokingStatus,Week_passed
0,ID00007637202177411956430,-4,2315,ID00007637202177411956430_-4,-4,2315,58.253649,79,Male,Ex-smoker,0
1,ID00007637202177411956430,5,2214,ID00007637202177411956430_5,-4,2315,58.253649,79,Male,Ex-smoker,9
2,ID00007637202177411956430,7,2061,ID00007637202177411956430_7,-4,2315,58.253649,79,Male,Ex-smoker,11
3,ID00007637202177411956430,9,2144,ID00007637202177411956430_9,-4,2315,58.253649,79,Male,Ex-smoker,13
4,ID00007637202177411956430,11,2069,ID00007637202177411956430_11,-4,2315,58.253649,79,Male,Ex-smoker,15


In [35]:
train = output[output['Week_passed']!=0].reset_index(drop=True)
print(train.shape)
train.head()


(12144, 11)


,Patient,predict_Week,FVC,Patient_Week,base_Week,base_FVC,base_Percent,base_Age,Sex,SmokingStatus,Week_passed
0,ID00007637202177411956430,5,2214,ID00007637202177411956430_5,-4,2315,58.253649,79,Male,Ex-smoker,9
1,ID00007637202177411956430,7,2061,ID00007637202177411956430_7,-4,2315,58.253649,79,Male,Ex-smoker,11
2,ID00007637202177411956430,9,2144,ID00007637202177411956430_9,-4,2315,58.253649,79,Male,Ex-smoker,13
3,ID00007637202177411956430,11,2069,ID00007637202177411956430_11,-4,2315,58.253649,79,Male,Ex-smoker,15
4,ID00007637202177411956430,17,2101,ID00007637202177411956430_17,-4,2315,58.253649,79,Male,Ex-smoker,21


## Construct Test Input

In [36]:
test = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')\
        .rename(columns={'Weeks': 'base_Week', 'FVC': 'base_FVC', 'Percent': 'base_Percent', 'Age': 'base_Age'})
submission = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/sample_submission.csv')
submission['Patient'] = submission['Patient_Week'].apply(lambda x: x.split('_')[0])
submission['predict_Week'] = submission['Patient_Week'].apply(lambda x: x.split('_')[1]).astype(int)
test = submission.drop(columns=['FVC', 'Confidence']).merge(test, on='Patient')
test['Week_passed'] = test['predict_Week'] - test['base_Week']
print(test.shape)
test.head()

(730, 10)


,Patient_Week,Patient,predict_Week,base_Week,base_FVC,base_Percent,base_Age,Sex,SmokingStatus,Week_passed
0,ID00419637202311204720264_-12,ID00419637202311204720264,-12,6,3020,70.186855,73,Male,Ex-smoker,-18
1,ID00419637202311204720264_-11,ID00419637202311204720264,-11,6,3020,70.186855,73,Male,Ex-smoker,-17
2,ID00419637202311204720264_-10,ID00419637202311204720264,-10,6,3020,70.186855,73,Male,Ex-smoker,-16
3,ID00419637202311204720264_-9,ID00419637202311204720264,-9,6,3020,70.186855,73,Male,Ex-smoker,-15
4,ID00419637202311204720264_-8,ID00419637202311204720264,-8,6,3020,70.186855,73,Male,Ex-smoker,-14


In [37]:
submission = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/sample_submission.csv')
print(submission.shape)
submission.head()

(730, 3)


,Patient_Week,FVC,Confidence
0,ID00419637202311204720264_-12,2000,100
1,ID00421637202311550012437_-12,2000,100
2,ID00422637202311677017371_-12,2000,100
3,ID00423637202312137826377_-12,2000,100
4,ID00426637202313170790466_-12,2000,100


# Modeling - Tabular data only

## Prepare folds

In [38]:
group_k_fold = GroupKFold(n_splits=N_FOLD)
groups = train['Patient'].values
for fold, (train_index, valid_index) in enumerate(group_k_fold.split(train, train[TARGET], groups)):
    train.loc[valid_index, 'fold'] = int(fold)
train['fold'] = train['fold'].astype(int)
train.head()

,Patient,predict_Week,FVC,Patient_Week,base_Week,base_FVC,base_Percent,base_Age,Sex,SmokingStatus,Week_passed,fold
0,ID00007637202177411956430,5,2214,ID00007637202177411956430_5,-4,2315,58.253649,79,Male,Ex-smoker,9,2
1,ID00007637202177411956430,7,2061,ID00007637202177411956430_7,-4,2315,58.253649,79,Male,Ex-smoker,11,2
2,ID00007637202177411956430,9,2144,ID00007637202177411956430_9,-4,2315,58.253649,79,Male,Ex-smoker,13,2
3,ID00007637202177411956430,11,2069,ID00007637202177411956430_11,-4,2315,58.253649,79,Male,Ex-smoker,15,2
4,ID00007637202177411956430,17,2101,ID00007637202177411956430_17,-4,2315,58.253649,79,Male,Ex-smoker,21,2


## XGB Model  (Hyperopt)

In [39]:
# features
target = train[TARGET]
test[TARGET] = np.nan
cat_features = ['Sex', 'SmokingStatus']
num_features = [c for c in test.columns if (test.dtypes[c] != 'object') & (c not in cat_features)]
features = num_features + cat_features
drop_features = [ID, TARGET, 'predict_Week', 'base_Week']
features = [c for c in features if c not in drop_features]

for col in cat_features:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(train[col])
    train.loc[:, col] = lbl.transform(train[col])
    test.loc[:, col] = lbl.transform(test[col])      

In [40]:
def optimize(params, x, y, groups):
    """
    The main optimizatiion function
    This function takes all the arguments from the search space and training features and target.
    It then initialized the models by setting the chosen paramaters and runs crossvalidation and returns a 
    negative accuracy score
    :param params: dict of params from hyperopt
    :param x: training data
    :param y: labels/targets
    :return negative accuracy after 5 folds
    """

    # initialize model with current parameters
    model = xgb.XGBRegressor(**params)

    group_k_fold = GroupKFold(n_splits=N_FOLD)

    # initialize accuracy list
    rmse_errors = []
    
    for idx in group_k_fold.split(x, y, groups):
        train_idx, test_idx = idx[0], idx[1]
        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        # fit model on train data
        model.fit(xtrain, ytrain)

        # create predictions
        preds = model.predict(xtest)

        fold_rmse = np.sqrt(mean_squared_error(ytest, preds))
        rmse_errors.append(fold_rmse)
    
    return np.mean(rmse_errors)


In [41]:
# param_space = {
        
#         'eta': hp.choice('eta', [0.01, 0.015, 0.025,0.05, 1]),    
#         'gamma': hp.choice('gamma', [0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.3, 0.5, 0.7, 0.9, 1]),
#         'max_depth':hp.choice('max_depth', [3,5,7,9,12,15,17,25]),
#         'min_child_weight': hp.choice('min_child_weight', [1,3,5,7]),
#         'subsample': hp.choice('subsample', [0.6, 0.7, 0.8, 0.9, 1.0]),
#         'colsample_bytree': hp.choice('colsample_bytree', [0.6, 0.7, 0.8, 0.9, 0.1]),
#         'objective':'reg:squarederror',
#         'eval_metric': 'rmse',
#     }


param_space = {
        'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
        'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
        'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
        'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
    }

groups = train.Patient.values
X = train[features].values
y = train[TARGET].values
optimization_function = partial(optimize, x=X, y=y, groups = groups)
# initialize Trials to keep logging information
trials = Trials()

# run hyperopt
hopt = fmin(fn=optimization_function,
space=param_space,
algo=tpe.suggest,
max_evals=50,
trials=trials)

print(hopt)

100%|██████████| 50/50 [00:44<00:00,  1.12trial/s, best loss: 254.7971011671423] 
{'colsample_bytree': 0.8500000000000001, 'eta': 0.225, 'gamma': 0.5, 'max_depth': 0, 'min_child_weight': 6.0, 'subsample': 0.55}


In [42]:
best_params = {'colsample_bytree': 1.0, 
               'eta': 0.1, 'gamma': 0.75, 
               'max_depth': 1, 'min_child_weight': 5.0, 'subsample': 0.65,
               'objective': 'reg:squarederror',
        'eval_metric': 'rmse'
              }
best_model = xgb.XGBRegressor(**best_params)

In [43]:
best_model.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1.0, eta=0.1,
             eval_metric='rmse', gamma=0.75, gpu_id=-1, importance_type='gain',
             interaction_constraints='', learning_rate=0.100000001,
             max_delta_step=0, max_depth=1, min_child_weight=5.0, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, subsample=0.65, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [44]:
preds = best_model.predict(test[features].values)

In [45]:
train['FVC_pred'] = best_model.predict(train[features].values)
test['FVC_pred'] = preds

In [26]:
# baseline score
train['Confidence'] = 100
train['sigma_clipped'] = train['Confidence'].apply(lambda x: max(x, 70))
train['diff'] = abs(train['FVC'] - train['FVC_pred'])
train['delta'] = train['diff'].apply(lambda x: min(x, 1000))
train['score'] = -math.sqrt(2)*train['delta']/train['sigma_clipped'] - np.log(math.sqrt(2)*train['sigma_clipped'])
score = train['score'].mean()
print(score)

-7.171272651567262


In [27]:
import scipy as sp

def loss_func(weight, row):
    confidence = weight
    sigma_clipped = max(confidence, 70)
    diff = abs(row['FVC'] - row['FVC_pred'])
    delta = min(diff, 1000)
    score = -math.sqrt(2)*delta/sigma_clipped - np.log(math.sqrt(2)*sigma_clipped)
    return -score

results = []
tk0 = tqdm(train.iterrows(), total=len(train))
for _, row in tk0:
    loss_partial = partial(loss_func, row=row)
    weight = [100]
    #bounds = [(70, 100)]
    #result = sp.optimize.minimize(loss_partial, weight, method='SLSQP', bounds=bounds)
    result = sp.optimize.minimize(loss_partial, weight, method='SLSQP')
    x = result['x']
    results.append(x[0])

In [49]:
# optimized score
train['Confidence'] = results
train['sigma_clipped'] = train['Confidence'].apply(lambda x: max(x, 70))
train['diff'] = abs(train['FVC'] - train['FVC_pred'])
train['delta'] = train['diff'].apply(lambda x: min(x, 1000))
train['score'] = -math.sqrt(2)*train['delta']/train['sigma_clipped'] - np.log(math.sqrt(2)*train['sigma_clipped'])
score = train['score'].mean()
print(score)
train.head()

-6.359231166303251


,Patient,predict_Week,FVC,Patient_Week,base_Week,base_FVC,base_Percent,base_Age,Sex,SmokingStatus,Week_passed,fold,FVC_pred,Confidence,sigma_clipped,diff,delta,score
0,ID00007637202177411956430,5,2214,ID00007637202177411956430_5,-4,2315,58.253649,79,1,1,9,2,2262.904297,49.949527,70.000000,48.904297,48.904297,-5.583085
1,ID00007637202177411956430,7,2061,ID00007637202177411956430_7,-4,2315,58.253649,79,1,1,11,2,2262.904297,285.521591,285.521591,201.904297,201.904297,-7.000941
2,ID00007637202177411956430,9,2144,ID00007637202177411956430_9,-4,2315,58.253649,79,1,1,13,2,2251.105957,151.363780,151.363780,107.105957,107.105957,-6.366966
3,ID00007637202177411956430,11,2069,ID00007637202177411956430_11,-4,2315,58.253649,79,1,1,15,2,2251.105957,257.411762,257.411762,182.105957,182.105957,-6.897736
4,ID00007637202177411956430,17,2101,ID00007637202177411956430_17,-4,2315,58.253649,79,1,1,21,2,2251.105957,212.272056,212.272056,150.105957,150.105957,-6.704489


## predict Confidence¶


In [52]:
TARGET = 'Confidence'

target = train[TARGET]
test[TARGET] = np.nan

# features
cat_features = ['Sex', 'SmokingStatus']
num_features = [c for c in test.columns if (test.dtypes[c] != 'object') & (c not in cat_features)]
features = num_features + cat_features
drop_features = [ID, TARGET, 'predict_Week', 'base_Week', 'FVC', 'FVC_pred']
features = [c for c in features if c not in drop_features]

In [53]:
features

['base_FVC', 'base_Percent', 'base_Age', 'Week_passed', 'Sex', 'SmokingStatus']

In [54]:
groups = train.Patient.values
X = train[features].values
y = train[TARGET].values
optimization_function = partial(optimize, x=X, y=y, groups = groups)
# initialize Trials to keep logging information
trials = Trials()

# run hyperopt
hopt = fmin(fn=optimization_function,
space=param_space,
algo=tpe.suggest,
max_evals=50,
trials=trials)

print(hopt)

100%|██████████| 50/50 [00:45<00:00,  1.09trial/s, best loss: 212.98512869412372]
{'colsample_bytree': 0.8, 'eta': 0.05, 'gamma': 0.8500000000000001, 'max_depth': 0, 'min_child_weight': 4.0, 'subsample': 0.8}


In [57]:
best_params = {'colsample_bytree': 0.8, 'eta': 0.05, 
               'gamma': 0.8500000000000001,
               'max_depth': 0,
               'min_child_weight': 4.0,
               'subsample': 0.8,
               'objective': 'reg:squarederror',
        'eval_metric': 'rmse'
              }
best_model = xgb.XGBRegressor(**best_params)
best_model.fit(X, y)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eta=0.05,
             eval_metric='rmse', gamma=0.8500000000000001, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.0500000007, max_delta_step=0, max_depth=0,
             min_child_weight=4.0, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [58]:
train['Confidence'] = best_model.predict(train[features].values)
train['sigma_clipped'] = train['Confidence'].apply(lambda x: max(x, 70))
train['diff'] = abs(train['FVC'] - train['FVC_pred'])
train['delta'] = train['diff'].apply(lambda x: min(x, 1000))
train['score'] = -math.sqrt(2)*train['delta']/train['sigma_clipped'] - np.log(math.sqrt(2)*train['sigma_clipped'])
score = train['score'].mean()
print(score)

-6.7490452706400115


In [61]:
def lb_metric(train):
    train['Confidence'] = best_model.predict(train[features].values)
    train['sigma_clipped'] = train['Confidence'].apply(lambda x: max(x, 70))
    train['diff'] = abs(train['FVC'] - train['FVC_pred'])
    train['delta'] = train['diff'].apply(lambda x: min(x, 1000))
    train['score'] = -math.sqrt(2)*train['delta']/train['sigma_clipped'] - np.log(math.sqrt(2)*train['sigma_clipped'])
    score = train['score'].mean()
    return score

In [62]:
score = lb_metric(train)
logger.info(f'Local Score: {score}')

Local Score: -6.7490452706400115
Local Score: -6.7490452706400115


In [63]:
test['Confidence'] = best_model.predict(test[features].values)

In [65]:
submission.head()

,Patient_Week,FVC,Confidence
0,ID00419637202311204720264_-12,2000,100
1,ID00421637202311550012437_-12,2000,100
2,ID00422637202311677017371_-12,2000,100
3,ID00423637202312137826377_-12,2000,100
4,ID00426637202313170790466_-12,2000,100


In [66]:
sub = submission.drop(columns=['FVC', 'Confidence']).merge(test[['Patient_Week', 'FVC_pred', 'Confidence']], 
                                                           on='Patient_Week')
sub.columns = submission.columns
sub.to_csv('submission.csv', index=False)
sub.head()

array([222.7575], dtype=float32)